In [12]:
!pip install -q pythainlp scikit-learn pandas matplotlib seaborn nltk joblib
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
import pandas as pd

csv_path = '/content/ReviewKruaBanPaBoon.csv'
df = pd.read_csv(csv_path)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()

Shape: (150, 3)
Columns: ['ชื่อ', 'ระยะเวลา', 'ข้อความรีวิว']


,ชื่อ,ระยะเวลา,ข้อความรีวิว
0,rattanaporn ratree,สัปดาห์ที่แล้ว,ดีทุกอย่าง วันที่มาเป็นเสาร์ เวลาราว 15.00น. ค...
1,Bow Khun,2 สัปดาห์ที่แล้ว,บรรยากาศดีมากกกกกกกก สาขานี้กว้างมาก ที่จอดรถเ...
2,soontaree pree,2 สัปดาห์ที่แล้ว,อาหารอร่อย บรรยากาศดีมาก เหมาะกับครอบครัวที่มี...
3,Natthaporn (nnxthq_),สัปดาห์ที่แล้ว,อาหารรสชาติอร่อย เข้มข้น บรรยากาศดี มีที่จอดรถ...
4,Panisa Kasemsopa,สัปดาห์ที่แล้ว,ร้านอาหารที่ร่มรื่น สะอาด อาหารอร่อย ทำถึงและต...


In [17]:
# === เซลล์ 1: ติดตั้ง และ import ไลบรารี ===
!pip install pythainlp

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import make_pipeline
import re

# === เซลล์ 2: อ่านไฟล์ CSV และตรวจคอลัมน์ ===
path = "/content/ReviewKruaBanPaBoon.csv"
df = pd.read_csv(path, encoding='utf-8', dtype=str, engine='python').fillna("")
print("Columns:", list(df.columns))
display(df.head(8))

# === เซลล์ 3: weak-label ===
positive_words = ["อร่อย","ดีมาก","ดี","ชอบ","อร่อยมาก","แนะนำ","อร่อยสุด","สด","ถูกใจ","ประทับใจ","เด็ด","คุ้ม","ถูก"]
negative_words = ["ไม่อร่อย","แย่","แพง","ไม่ดี","เสียใจ","ผิดหวัง","บริการแย่","รสจืด","ห่วย","ไม่โอเค","ไม่ประทับใจ","สกปรก","เก่า"]

def weak_label(text):
    t = str(text).lower()
    pos = any(w in t for w in positive_words)
    neg = any(w in t for w in negative_words)
    if pos and not neg:
        return "positive"
    elif neg and not pos:
        return "negative"
    else:
        return "neutral"

# สร้าง column weak_label จากคอลัมน์ 'ข้อความรีวิว'
source_col = "ข้อความรีวิว"
df['weak_label'] = df[source_col].apply(weak_label)
print(df['weak_label'].value_counts())

# === เซลล์ 4: สร้าง pipeline TF-IDF (char ngram) + MultinomialNB และ cross-val ===
vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(1,4), max_features=20000)
pipeline = make_pipeline(vectorizer, MultinomialNB())

X = df[source_col].astype(str)
y = df['weak_label']

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
try:
    scores = cross_val_score(pipeline, X, y, cv=skf, scoring='accuracy')
    print(f"Cross-val accuracy (mean over {n_splits} folds): {scores.mean():.4f}")
except Exception as e:
    print("Cross-validation failed (probably due to very imbalanced/rare classes). Exception:", e)

pipeline.fit(X, y)

# === เซลล์ 5: ทำนาย sentiment และบันทึกผล ===
df['predicted_sentiment'] = pipeline.predict(X)
display(df.head(10))

out_path = "/content/ReviewKruaBanPaBoon_with_sentiment.csv"
df.to_csv(out_path, index=False, encoding='utf-8-sig')
print("Saved annotated CSV to:", out_path)

# === เซลล์ 6: สรุปการกระจาย sentiment และตัวอย่างข้อความที่เป็นแต่ละกลุ่ม ===
print("\nWeak-label distribution:")
print(df['weak_label'].value_counts())
print("\nPredicted sentiment distribution:")
print(df['predicted_sentiment'].value_counts())

# แสดงตัวอย่าง 5 ตัวอย่างจากแต่ละกลุ่ม
for lab in ['positive','neutral','negative']:
    subset = df[df['predicted_sentiment']==lab]
    if len(subset)>0:
        print(f"\n=== Example predicted {lab} (up to 5) ===")
        display(subset[[source_col, 'predicted_sentiment']].head(5))

Columns: ['ชื่อ', 'ระยะเวลา', 'ข้อความรีวิว']


,ชื่อ,ระยะเวลา,ข้อความรีวิว
0,rattanaporn ratree,สัปดาห์ที่แล้ว,ดีทุกอย่าง วันที่มาเป็นเสาร์ เวลาราว 15.00น. ค...
1,Bow Khun,2 สัปดาห์ที่แล้ว,บรรยากาศดีมากกกกกกกก สาขานี้กว้างมาก ที่จอดรถเ...
2,soontaree pree,2 สัปดาห์ที่แล้ว,อาหารอร่อย บรรยากาศดีมาก เหมาะกับครอบครัวที่มี...
3,Natthaporn (nnxthq_),สัปดาห์ที่แล้ว,อาหารรสชาติอร่อย เข้มข้น บรรยากาศดี มีที่จอดรถ...
4,Panisa Kasemsopa,สัปดาห์ที่แล้ว,ร้านอาหารที่ร่มรื่น สะอาด อาหารอร่อย ทำถึงและต...
5,Minnim Aorranoy,สัปดาห์ที่แล้ว,ร้านป้าบุญเคยไปสาขาอื่น บรรยากาศเหมือนกันเลย จ...
6,Suthasinee Trithara,3 สัปดาห์ที่แล้ว,เป็นร้านที่อยากมานานแล้ว แต่ได้ยินว่าคิวยาว วั...
7,nongnapat preechakiat,สัปดาห์ที่แล้ว,อาหารมาเร็ว รสชาติอร่อย อาหารจานใหญ่สมกับราคาเ...


weak_label
positive    128
neutral      21
negative      1
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-val accuracy (mean over 5 folds): 0.8533


,ชื่อ,ระยะเวลา,ข้อความรีวิว,weak_label,predicted_sentiment
0,rattanaporn ratree,สัปดาห์ที่แล้ว,ดีทุกอย่าง วันที่มาเป็นเสาร์ เวลาราว 15.00น. ค...,positive,positive
1,Bow Khun,2 สัปดาห์ที่แล้ว,บรรยากาศดีมากกกกกกกก สาขานี้กว้างมาก ที่จอดรถเ...,positive,positive
2,soontaree pree,2 สัปดาห์ที่แล้ว,อาหารอร่อย บรรยากาศดีมาก เหมาะกับครอบครัวที่มี...,positive,positive
3,Natthaporn (nnxthq_),สัปดาห์ที่แล้ว,อาหารรสชาติอร่อย เข้มข้น บรรยากาศดี มีที่จอดรถ...,positive,positive
4,Panisa Kasemsopa,สัปดาห์ที่แล้ว,ร้านอาหารที่ร่มรื่น สะอาด อาหารอร่อย ทำถึงและต...,positive,positive
5,Minnim Aorranoy,สัปดาห์ที่แล้ว,ร้านป้าบุญเคยไปสาขาอื่น บรรยากาศเหมือนกันเลย จ...,positive,positive
6,Suthasinee Trithara,3 สัปดาห์ที่แล้ว,เป็นร้านที่อยากมานานแล้ว แต่ได้ยินว่าคิวยาว วั...,positive,positive
7,nongnapat preechakiat,สัปดาห์ที่แล้ว,อาหารมาเร็ว รสชาติอร่อย อาหารจานใหญ่สมกับราคาเ...,positive,positive
8,Arunothai Jantarachot,สัปดาห์ที่แล้ว,เริ่ดๆๆ มีน้องสัตว์ต่างๆ สนามเด็กเล่น ไดโนเสาร...,positive,positive
9,Ananya Wichapha,เดือนที่แล้ว,เต็ม10ให้100 ไปกับครอบครัวคือดีมาก ของเล่นเด็ก...,positive,positive


Saved annotated CSV to: /content/ReviewKruaBanPaBoon_with_sentiment.csv

Weak-label distribution:
weak_label
positive    128
neutral      21
negative      1
Name: count, dtype: int64

Predicted sentiment distribution:
predicted_sentiment
positive    150
Name: count, dtype: int64

=== Example predicted positive (up to 5) ===


,ข้อความรีวิว,predicted_sentiment
0,ดีทุกอย่าง วันที่มาเป็นเสาร์ เวลาราว 15.00น. ค...,positive
1,บรรยากาศดีมากกกกกกกก สาขานี้กว้างมาก ที่จอดรถเ...,positive
2,อาหารอร่อย บรรยากาศดีมาก เหมาะกับครอบครัวที่มี...,positive
3,อาหารรสชาติอร่อย เข้มข้น บรรยากาศดี มีที่จอดรถ...,positive
4,ร้านอาหารที่ร่มรื่น สะอาด อาหารอร่อย ทำถึงและต...,positive
